In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from experiment_tools import *

In [ ]:
data_config = DataConfig(path = 'data')
dt = DataTools(data_config)

In [ ]:
experiment = ModelExperiment(experiment_name='pix2struct_base', 
                             data_config = data_config,
                             )

In [ ]:
dataset = dt.build_dataset(stage='dev', sample_size=20, train_val_split=0.8)
train_data, val_data = dataset['train'], dataset['val']

In [ ]:
generator = CaptionGenerator(device='mps')

In [ ]:
generator.load_model()

In [ ]:
experiment.train_model(generator = generator,
                       train_dataset = train_data,
                       epochs = 1,
                       batch_size = 2)